#### Build a regression model.



Building a regression model to demonstrate a relationship between the number of bikes and average number of reviews of nearby restaurants and average rating of nearby restaurants

In [8]:
import sqlite3 as sqlite
import pandas as pd
from sqlalchemy import create_engine

connection = create_engine('sqlite:///bike_and_restaurants.db')
    
sql = ''' SELECT      b.station_name,
                      number_of_bikes,
                      AVG(review_count) AS avg_review_count,
                      AVG(rating) AS avg_rating
          FROM        bike_stations b
          JOIN        nearby_restaurants nr
          ON          b.station_name = nr.station_name
          JOIN        restaurants r
          ON          nr.restaurant_name = r.restaurant_name
          WHERE       review_count IS NOT NULL
          GROUP BY    b.station_name, number_of_bikes '''

df = pd.read_sql_query(sql, connection)

#### Provide model output and an interpretation of the results. 

In [12]:
import statsmodels.api as sm

y = df['number_of_bikes']
x = df[['avg_review_count', 'avg_rating']]

lin_reg = sm.OLS(y, x)

model = lin_reg.fit()
regression_model = model.summary()
print(regression_model)

                                 OLS Regression Results                                
Dep. Variable:        number_of_bikes   R-squared (uncentered):                   0.929
Model:                            OLS   Adj. R-squared (uncentered):              0.919
Method:                 Least Squares   F-statistic:                              91.18
Date:                Mon, 15 Apr 2024   Prob (F-statistic):                    9.37e-09
Time:                        12:47:23   Log-Likelihood:                         -38.683
No. Observations:                  16   AIC:                                      81.37
Df Residuals:                      14   BIC:                                      82.91
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                       coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------

/opt/homebrew/Caskroom/miniforge/base/envs/base_env/lib/python3.8/site-packages/scipy/stats/_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


Removed review_count due to the low p-value in order to get the best of fit model

In [11]:
x = df['avg_rating']

lin_reg = sm.OLS(y, x)

model = lin_reg.fit()
print_model = model.summary()
print(print_model)

                                 OLS Regression Results                                
Dep. Variable:        number_of_bikes   R-squared (uncentered):                   0.928
Model:                            OLS   Adj. R-squared (uncentered):              0.924
Method:                 Least Squares   F-statistic:                              194.2
Date:                Sun, 14 Apr 2024   Prob (F-statistic):                    5.45e-10
Time:                        15:36:59   Log-Likelihood:                         -38.726
No. Observations:                  16   AIC:                                      79.45
Df Residuals:                      15   BIC:                                      80.22
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

/opt/homebrew/Caskroom/miniforge/base/envs/base_env/lib/python3.8/site-packages/scipy/stats/_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


From the regression model it can be inferred that there is a positive correlation between the number of bikes in a station and the average rating of restaurants nearby that station. Both the low p-value and the high Adj. R-squared indicates that the model is a good fit. There is also a low Prob (F-statistic) meaning we can reject the null hypothesis (that there is not a correlation). Although there is an error at the bottom that suggests the model may not be as reliable due to the small sample size.

# Stretch

#### How can you turn the regression model into a classification model?

It is possible to change the regression model into a classification model by splitting the number_of_bikes into either 'high' or 'low'. This can be done by changing the datatype into object, or categorical type. As well as changing the number of bikes in a station. If higher than the average number of bikes in the column then changed into 'high' and if lower than average changed into 'low'.

Then using this new number of bikes column and the rating/review counts, its possible to create a classification problem where the higher/lower than average number of bikes is correlated to the ratings/review counts.